# Data exploration, cleaning and manipulation

In [99]:
import pandas as pd 
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [100]:
albums = pd.read_csv('albums.csv', index_col=0, parse_dates=True)[1:]
albums['date'] = pd.to_datetime(albums.date)
print(albums.shape)
albums.head(3)

(573946, 7)


,id,date,artist,album,rank,length,track_length
index,,,,,,,
1,2,2019-01-19,A Boogie Wit da Hoodie,Hoodie SZN,1.0,20.0,185233.800000
2,3,2019-01-19,21 Savage,I Am > I Was,2.0,15.0,211050.733333
3,4,2019-01-19,Soundtrack,Spider-Man: Into The Spider-Verse,3.0,13.0,190866.384615


Albums sorted by # weeks in the Billboard Top 200


In [101]:
albums.groupby(['album', 'artist'])['rank'].count().sort_values(ascending=False).head(20)

album                                 artist                                             
The Dark Side Of The Moon             Pink Floyd                                             941
Legend: The Best Of...                Bob Marley And The Wailers                             555
Journey's Greatest Hits               Journey                                                545
Metallica                             Metallica                                              516
Greatest Hits                         Guns N' Roses                                          451
Curtain Call: The Hits                Eminem                                                 426
Nevermind                             Nirvana                                                406
Doo-Wops & Hooligans                  Bruno Mars                                             404
Chronicle The 20 Greatest Hits        Creedence Clearwater Revival Featuring John Fogerty    400
21                                   

In [102]:
acoustic_features = pd.read_csv('acoustic_features.csv', index_col=0)
print(acoustic_features.shape)
acoustic_features.head(3)

(339855, 19)


,id,song,album,artist,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,album_id,date
index,,,,,,,,,,,,,,,,,,,
0,0Veyvc3n9AcLSoK3r1dA12,Voices In My Head,Hoodie SZN,A Boogie Wit da Hoodie,0.0555,0.754,142301.0,0.663,0.000000,6.0,0.101,-6.311,0.0,0.427,90.195,4.0,0.207,3r5hf3Cj3EMh1C2saQ8jyt,2018-12-21
1,77JzXZonNumWsuXKy9vr3U,Beasty,Hoodie SZN,A Boogie Wit da Hoodie,0.2920,0.860,152829.0,0.418,0.000000,7.0,0.106,-9.061,0.0,0.158,126.023,4.0,0.374,3r5hf3Cj3EMh1C2saQ8jyt,2018-12-21
2,18yllZD0TdF7ykcREib8Z1,I Did It,Hoodie SZN,A Boogie Wit da Hoodie,0.1530,0.718,215305.0,0.454,0.000046,8.0,0.116,-9.012,1.0,0.127,89.483,4.0,0.196,3r5hf3Cj3EMh1C2saQ8jyt,2018-12-21


Cleaning acoustic features

In [103]:
# Remove tracks less than a minute long 
features = acoustic_features.loc[acoustic_features['duration_ms'] > 60000]
# Drop non numeric columns 
features.drop(columns=['id', 'song', 'artist', 'album_id', 'date'], inplace=True)
# Extract total run time per album 
length = features.groupby('album')['duration_ms'].sum()
# Set index to album  
features.set_index('album', inplace=True)
# Set length name to weights for clarity in weighted mean calculation
length.name = 'weights'
features.head(3)

C:\Users\shaiyon\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
album,,,,,,,,,,,,,
Hoodie SZN,0.0555,0.754,142301.0,0.663,0.000000,6.0,0.101,-6.311,0.0,0.427,90.195,4.0,0.207
Hoodie SZN,0.2920,0.860,152829.0,0.418,0.000000,7.0,0.106,-9.061,0.0,0.158,126.023,4.0,0.374
Hoodie SZN,0.1530,0.718,215305.0,0.454,0.000046,8.0,0.116,-9.012,1.0,0.127,89.483,4.0,0.196


Aggregating songs by album weighted by the percentage of the album they take up. 

In [104]:
features = features.join(length)
# Extract percentage of the album each song takes up 
features['weights'] = features['duration_ms'] / features['weights']
# Multiply weights by each column 
features = features.drop(['weights', 'duration_ms'], axis=1).mul(features['weights'], axis=0)
length.name = 'length'
# Sum up the weighted data, grouping by album. Then add album length. 
features = features.groupby('album').sum().join(length)
features.head(3)

,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,length
album,,,,,,,,,,,,,
!!Going Places!!,0.504944,0.634360,0.497121,0.636872,5.469885,0.107299,-11.909669,0.578203,0.093018,115.741766,4.000000,0.636955,1786385.0
!Viva El Amor!,0.102661,0.507975,0.717777,0.002417,7.703102,0.165748,-5.041795,0.723996,0.037021,129.670102,3.850478,0.530391,2715761.0
!Viva La Cobra!,0.046689,0.586020,0.789629,0.000007,2.522854,0.242735,-5.825477,0.282722,0.071973,122.389089,3.701332,0.700530,2219053.0


Next, we'll center and scale the features to have a mean of 0 and unit variance to make future regression coefficients significantly more interpretable.

In [105]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
features = pd.DataFrame(ss.fit_transform(features), columns=features.columns, index=features.index)
features.head(3)

,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,length
album,,,,,,,,,,,,,
!!Going Places!!,0.918999,0.732403,-0.668920,2.949217,0.184260,-0.789433,-0.772536,-0.611254,0.090508,-0.380813,0.612909,0.810201,-0.822032
!Viva El Amor!,-0.744326,-0.263155,0.523593,-0.519468,1.953606,-0.384624,1.027195,0.094299,-0.500926,0.713587,-0.306337,0.175245,-0.282002
!Viva La Cobra!,-0.975756,0.351619,0.911908,-0.532645,-2.150631,0.148572,0.821831,-2.041211,-0.131773,0.141491,-1.223267,1.189003,-0.570623


In [116]:
reviews = pd.read_csv('reviews.csv').drop(['index','id','role'], axis=1)
reviews.set_index('album', inplace=True)
print(reviews.shape)
reviews.head(3)

(20873, 8)


,artist,genre,score,date,author,review,bnm,link
album,,,,,,,,
“…The Best Live Show of All Time” — NME EP,David Byrne,Rock,5.5,January 11 2019,Andy Beta,"Viva Brother, Terris, Mansun, the Twang, Joe L...",0,https://pitchfork.com/reviews/albums/david-byr...
Lost Lovesongs / Lostsongs Vol. 2,DJ Healer,Electronic,6.2,January 11 2019,Chal Ravens,"The Prince of Denmark—that is, the proper prin...",0,https://pitchfork.com/reviews/albums/dj-healer...
Roman Birds,Jorge Velez,Electronic,7.9,January 10 2019,Philip Sherburne,"Jorge Velez has long been prolific, but that’s...",0,https://pitchfork.com/reviews/albums/jorge-vel...


In [117]:
print(reviews['genre'].nunique())
reviews['genre'].unique()[0:15]

137


array(['Rock', 'Electronic', 'Rap', 'Experimental', 'Pop/R&B',
       'Folk/Country', nan, 'Metal', 'Rock,Pop/R&B', 'Jazz',
       'Rock,Experimental', 'Metal,Rock', 'Jazz,Experimental',
       'Experimental,Electronic', 'Folk/Country,Experimental'],
      dtype=object)

We want to regress the data separately for each genre, but 137 is simply too many genres to deal with. So we'll use a regular expression to simplify the strings in the genre column to be only the first/primary genre. This puts the albums into broader, more easily digestable categories.

In [118]:
reviews['genre'] = reviews['genre'].replace(r'(,|/)[\w\W]+', '', regex=True)
print(reviews['genre'].nunique())
reviews['genre'].unique()

9


array(['Rock', 'Electronic', 'Rap', 'Experimental', 'Pop', 'Folk', nan,
       'Metal', 'Jazz', 'Global'], dtype=object)

Now that the data is fully cleaned, it's time to join the features and reviews dataframes, removing albums that don't have review data in the process.

In [119]:
# Join features and reviews, remove albums that don't have review data. 
data = features.join(reviews).dropna()
print(data.shape)
data.head(3)

(2948, 21)


,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,...,valence,length,artist,genre,score,date,author,review,bnm,link
album,,,,,,,,,,,,,,,,,,,,,
$O$,-0.712969,1.616222,0.483327,-0.141289,-0.461014,-0.283236,0.529720,-0.764353,1.526188,0.313339,...,0.171074,0.019912,Die Antwoord,Rap,5.5,October 20 2010,Scott Plagenhoef,Die Antwoord were always a group of musicians ...,0.0,https://pitchfork.com/reviews/albums/14766-o/
...And Star Power,-0.330429,-1.447793,-0.062838,0.952225,-0.083678,0.424758,0.117350,1.024819,-0.234753,0.316331,...,-0.268539,0.991286,Foxygen,Rock,7.0,October 13 2014,Stuart Berman,Embedded within the detailed credits to Foxyge...,0.0,https://pitchfork.com/reviews/albums/19769-fox...
...And Then You Shoot Your Cousin,0.557710,0.576230,-0.519751,-0.352142,-0.337685,-0.670624,-0.699499,-1.938464,0.914589,-0.968447,...,-0.788879,-0.718222,The Roots,Rap,7.2,May 23 2014,Jayson Greene,"""Yes, @TheRoots have NEVER been conventional i...",0.0,https://pitchfork.com/reviews/albums/19332-the...


# Linear Regression on acoustic features and Pitchfork score 

In [120]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse

# Iterate through genres and regress with sklearn.
for genre in data['genre'].unique():
    genre_df = data.loc[data['genre'] == genre]
    X = genre_df.iloc[:,:13]
    y = genre_df['score']
    reg = LinearRegression()
    reg.fit(X, y)
    preds = reg.predict(X)
    print(genre + ' R\N{SUPERSCRIPT TWO}: ' + format(reg.score(X, y)) 
    + ', RMSE: ' + format(np.sqrt(mse(y, preds))) + ', sample size: ' + format(len(X)))

Rap R²: 0.07288157485061086, RMSE: 1.3581790455032696, sample size: 504
Rock R²: 0.0797062912408657, RMSE: 1.46150652651079, sample size: 1221
Experimental R²: 0.09782990267975689, RMSE: 0.9405368707038011, sample size: 155
Metal R²: 0.18477495178686165, RMSE: 1.365830044674201, sample size: 121
Pop R²: 0.06175771940415598, RMSE: 1.2648975367365076, sample size: 331
Folk R²: 0.16233746607656363, RMSE: 1.0382067170991376, sample size: 84
Electronic R²: 0.04155489632622955, RMSE: 1.528541225341864, sample size: 487
Global R²: 0.8940538812188007, RMSE: 0.3236186358967899, sample size: 17
Jazz R²: 0.5823102835215439, RMSE: 0.7381688720787419, sample size: 28


The results form the sklearn regression are pretty uninterpretable, except that it's seriously struggling to accurately predict the score. More statistically sound insight may be gained from using the statsmodels package.

In [121]:
import statsmodels.api as sm

for genre in data['genre'].unique():
    genre_df = data.loc[data['genre'] == genre]
    X = genre_df.iloc[:,:13]
    y = genre_df['score']
    ols = sm.OLS(y, X)
    res = ols.fit()
    print('\n'+format(genre))
    print(res.summary())


Rap
                            OLS Regression Results                            
Dep. Variable:                  score   R-squared:                       0.787
Model:                            OLS   Adj. R-squared:                  0.781
Method:                 Least Squares   F-statistic:                     139.4
Date:                Thu, 13 Jun 2019   Prob (F-statistic):          2.17e-155
Time:                        23:54:54   Log-Likelihood:                -1307.9
No. Observations:                 504   AIC:                             2642.
Df Residuals:                     491   BIC:                             2697.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
acousticness        -1.4163      0.


Global
                            OLS Regression Results                            
Dep. Variable:                  score   R-squared:                       0.936
Model:                            OLS   Adj. R-squared:                  0.729
Method:                 Least Squares   F-statistic:                     4.514
Date:                Thu, 13 Jun 2019   Prob (F-statistic):             0.0784
Time:                        23:54:54   Log-Likelihood:                -34.492
No. Observations:                  17   AIC:                             94.98
Df Residuals:                       4   BIC:                             105.8
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
acousticness        13.5162     

C:\Users\shaiyon\Anaconda3\lib\site-packages\scipy\stats\stats.py:1394: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=17
  "anyway, n=%i" % int(n))
